# Alice, Bob & Eve: Digital signatures using quantum-safe ML-DSA

The algorithms mentioned in previous parts (RSA, ECDSA) are not quantum-safe. Once Eva will have quantum computer available, she can forge the signatures. Luckily, we already have two post-quantum digital signature algorithms available. Here we'll take a look at the ML-DSA (Module‑Lattice‑based Digital Signature Algorithm), formerly known as CRYSTALS-Dilithium.

This algorithm is available natively in .NET 10. For older versions you can use the `Microsoft.Bcl.Cryptography` package. We will be actually doing this, because current version of .NET Interactive is using .NET 9.

In [ ]:
// Install the Microsoft.Bcl.Cryptography package (required for .NET < 10.0)
// #r "nuget: Microsoft.Bcl.Cryptography"

// Import namespace
using System.Security.Cryptography;

// Show what runtime we're actually on
Console.WriteLine($"Runtime version:  {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");

// Check whether ML-DSA is supported on this platform
Console.WriteLine($"ML-DSA supported: {MLDsa.IsSupported}");

Runtime version:  .NET 10.0.3
ML-DSA supported: True


## Alice

As usual, Alice will generate her key pair. Notice that the keys are significantly longer not only than ECDSA, but also than RSA keys. But this is completely different scheme, so it's quite hard to compare them.

The API is kind of similar to the one exposed by `AsymmetricAlgorithm` base class, but the post-quantum algorithm classes are not derived from this class, because the philosophy and usage is different enough.

The ML-DSA is available in three variants: ML-DSA-44, ML-DSA-65 and ML-DSA-87. The numbers are arbitrary; they are sequential numbers of original CRYSTALS-Dilithium parameter sets and do not directly relate to any specific parameters, such as key sizes.

Higher number is "better" in terms of security (and "worse" in terms of performance and data size). The following table shows the three variants and their parameters (security roughly equivalent to AES with given key lenghs and size of various aspects):

| Variant | AES Equivalent Length | Public Key Length (bytes) | Public Key Length (bits) | Private Key Length (bytes) | Private Key Length (bits) | Signature Length (bytes) | Signature Length (bits) |
|--------|---------------:|--------------------:|-------------------:|---------------------:|--------------------:|-------------------:|------------------:|
| MLDSA44 | 128 | 1312 | 10496 | 2528 | 20224 | 2420 | 19360 |
| MLDSA65 | 192 | 1952 | 15616 | 4000 | 32000 | 3293 | 26344 |
| MLDSA87 | 256 | 2592 | 20736 | 4864 | 38912 | 4595 | 36760 |

For this demo we will be using the simplest variant, ML-DSA-44:

In [2]:
using System.Security.Cryptography;

var mldsa = MLDsa.GenerateKey(MLDsaAlgorithm.MLDsa44);
var privateKeyBytes = mldsa.ExportMLDsaPrivateKey();
var publicKeyBytes = mldsa.ExportMLDsaPublicKey();


// Display the keys
Console.WriteLine($"Alice's Private Key ({privateKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(privateKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Private Key (20480 bits):
i014+9O2lyqDHDAHNJtKq9pGCvg0U6EE2pU4ichhpdvZ/l8myYROe6SDHNCqDknh3LCTEI5uB9Tx
WNcIWjkag3RLcEQxBH2H9WAvDCvOUYcoX/DvgSvaqvhZJvMecdz3rQTjGqNfemVt3HY3jlXR3xQT
71v/YpBkYO+sAPJwR2YiwoRREg0TCETBFIAgN4UER2ZSSEHCtIQDMWQBA0jAMABZlASDBiAkwEQM
QiWbKCpSGAgKACKQknAQQ2bbEGrTRAxkOGRDGDIDSCCIwmAgEClZwkQiMiokJi6Cki2kIAJJmIET
kZDQRIIRNoiTsCSQBIjDplAMNWUTqBHjMgDAsk3KFGIYB0SgqG3aEgTTRIokoi0RJ1JTEA0II2jY
GIqAho2UAGkIJCUTRAKSMCYEsyXDSIxANJASEUVDwGRIGGUYN2QIx4UKmAgCxihJJArYkiFcIAJB
uAmSKATgBkBZlmQJEE6jNILLuCAaAUATF4zUMiRTFCzCtigTJ4SEBGUUokiDlEABOC4JFYIYQHGJ
AgKECCGhgCHBCGKhoizjqInBNC4DiQGEBigEOVKUoARbQi1UpijTsikCAhGYiHGLEEaEsggSCYpD
BoVhsCALlyDItmiCAmmcSJHLQGZhuDGbFIUZwIxSyFAYKC0MFBAhJy0ksAiDBnEYJYAZiYWColCB
JnLahG1CyCUZqQFKEC3SwkQEKYyISA0TECkkoGEaOAIDMATABEyLsCiRGEFSRIhQqC2jAIzjAi7J
soiDAESAtgUiSCEjAyoENkmhNDAasGGSkEFKEGoQgIikwiALRm5EloHcgkwgIyibpgFapo3LxCVj
loEYRgkSpCEAAWqCJAoBl0HbhFADwWRDKAkJB46cCG0IRzFcsAQQpxDBJI1iAoUiRxDEqGAkRmwA
IUzjkIxYQEzBEILJoC2BMEikAoQJNE1QBGwBx42MMF

In [3]:
Console.WriteLine($"Alice's Public Key ({publicKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(publicKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Public Key (10496 bits):
i014+9O2lyqDHDAHNJtKq9pGCvg0U6EE2pU4ichhpduWy9VfQsKyVFaSxCB0kvucMNgB0gmaGIQU
KQ39DmXaNJC/zRQB8axdkf+4Ezw5OilX8wh+gzx+YzExe5b5pFSR7E+etZdYnx1piWnl8V5JkkKQ
ViRlSIsr4pd5IbbLrt6yS4aT0uYCMxfiq6hF4+ozxCUkZl5MHmIFO/f3zrRrAb248h8qjHuiAYJ8
e6PX77TPKpgO4gQDr/eaz8C77l/X2I7sDRBpPSWpzyrVtoGigSaQ5ub27rGwJcZHIYklEOuJ4AxC
dAzgguRFetBkI2XP8rAvLI47MGBjSaz8e1CJTVdEe1vXDyd/YJBQq5EEVumNyqXF/ePO1IWtBp5i
wGJBzIpaeZd9CeCcFXqjOtW6ztcD55j507l/W9FLO42xsaTkcdHzgFhizR/+H6B9lbIODVpFv8Zm
EsLvRZmn9xb45zuvTA3r/JbbLVPjqwQBtzJ2mxdPZ49Sb9t7Jr0NuzbQX/Rxo1/fKKA2fauQeGaZ
Lj99Tchyl4ccxWgukRkGkyqbyvNAaGcsDYtOQMYB8jGYEOiKavqP45wcHgpb9W7aBwvvOTgBX9pb
/w8BTp5600000ZjzAcVMfjmn+2OX96qYUhq1SFxFgOFKr2OvxmdTEC1oInUnsWMz0lvc7nT4b8Fq
aJwIB84nAoL+bHmRGUDWSs4imlNwTlXN0I8tAFp4Q5eW9xBjRh2cFujwq6j9jFbJrFD6Eo39JA9s
OpmKozUZJ6JIVb8BlBCYBgxd4ypLSAF15RFML6ZIU7xs+t0AYqbhBbJba1mr5zG3+Nm1oth1An0Z
HaKPnoE0GUHIkJQ1jx6QQw9dyTlVDVqu3XnuvCYyh4QVWTL5XI6wFjE5qK2GiWpaCF7qqmnO2XUv
Nnby9x6hmafziOFsmZHMy+awhgnRc6n/DYEPMhou1nM

Alice will then generate signature of her message. Notice that the signature itself is also larger than DSA or RSA:

In [4]:
var messageText = "Bob, I love you!";
var messageBytes = System.Text.Encoding.UTF8.GetBytes(messageText);
Console.WriteLine($"Original Message: {messageText}");
Console.WriteLine();

// Sign the message
var signatureBytes = mldsa.SignData(messageBytes);
var signatureText = Convert.ToBase64String(signatureBytes, Base64FormattingOptions.InsertLineBreaks);
Console.WriteLine($"Signature:");
Console.WriteLine(signatureText);

Original Message: Bob, I love you!

Signature:
qchKrAzSAMU5GUzbutSz0R+hIYOXeiW4Zz1X05HaFI34gaXOg9E6TMaffYXWNYom7PfPD3inDwfR
uLzOBKDGzjhOpa4fUyL/UlKMB/hYGgCeNHMxvQnMzxbUX9cgvj5oKC8zBjw+KmiisHDTvnorWu8a
Xj7G7G6yjhp4Nr4NBJbQG0XOy7srxDGVBqvR0PD5EjAFjLPQoRlKFT2FqNhHJQUrT4yrJmGI8a2s
5gdyWC5arHLs/PzAlO697t+WXY4ZIoF2MCE0CB2/VOktx7ddWmP2YG4zo1aFQTUO5jvicSeQMdM9
y8I0tWcOKJRWk44RuDO2+QzxpM3i/Mva9pp24UKWCUDBFZLZrJRRuPisVu5AdViRSXfONGgidvtD
aJfBhkYcSxiBufkx+J9b23HB/uBXiGDcdyhd3W6p4G7k9kubuZkmZnVEpzpqnbGeDbpMlDvqC0DC
m60epHOJOBkXiZXMcTIn7PNzxJbFYb7fwbbR633E/8tjUvRT19eGxNS5gPxPeG+56kCWNJXpK6Wz
VnLEPYQl3nBOSD3uL6jpB/pBIuXQYtZpmZwkjJbHltx1AFU6dRbZKhHM76cjbQYjwekJSk8xISV5
WmnnPVCRQT6oTG4qBoRysetziGsiyshiUFO8JmjXnf6VyERv859Tjynp2CTTegdRY3sJb716WkmZ
GLnmn6caRF1Q1b6Gso0M7i9ADlg1wKaAR2NfrH52tTI8IO2Rk+WW5SQKgT1EZm8mfiE3YaPjrp8K
r56+bBeoLx7S9BRs3UTwChq0vLZbvoxYTLJ5YKyTxkYN9gTPdDEPBM+kMXjmjPdY6FEzloDeDq1V
svL5LusqLXRwglDR9IlNfVO10zm/Si0ndpvxLCOJWodrvCc97nIi7hERtGuFXzFv4K5nPP13543m
LJt6f5616KxQija7tIYx3ZdW5tPd/

## Bob

Bob will verify the signature in a fashion similar to RSA or DSA:

In [5]:
// Function to verify signature
bool VerifySignature(byte[] publicKey, string message, byte[] signature) {
    // Create MLDsa instance for verification using the public key
    using var mldsaVerify = MLDsa.ImportMLDsaPublicKey(MLDsaAlgorithm.MLDsa44, publicKey);

    // Convert message to bytes
    var messageBytesVerify = System.Text.Encoding.UTF8.GetBytes(message);

    // Verify the signature
    return mldsaVerify.VerifyData(messageBytesVerify, signature);
}

Console.WriteLine($"Is 'Bob, I love you!' valid? {VerifySignature(publicKeyBytes, "Bob, I love you!", signatureBytes)}");
Console.WriteLine($"Is 'Bob, I hate you!' valid? {VerifySignature(publicKeyBytes, "Bob, I hate you!", signatureBytes)}");

Is 'Bob, I love you!' valid? True
Is 'Bob, I hate you!' valid? False
